In [1]:
from latentneural.lorenz import LorenzGenerator, ic
from latentneural.data import DataManager
from latentneural.utils import logger
import numpy as np
import tensorflow as tf
import os
import yaml
from copy import deepcopy

In [2]:
step = 0.01 # Time step in the simulatio (in seconds)
stop = 1 # Final time for the simulation (in seconds)
neurons = 30 # Number of neurons
base_rate = 5 # Base frequency (in Hz)
latent_dim = 3 # Lorenz latent variables encoded in neural activity (relevant+irrelevant)
relevant_dim = 2 # Lorenz latent variables encoded in behavioural activity (relevant)
behaviour_dim = 4 # Behavioural channels
conditions = 1 # Number of conditions to be tested (each condition has different weight matrices to convert latent variables into firing rates and behaviour)
trials = 1000 # Number of trials to be tested for each condition
initial_conditions = ic.uniform(low=-10, high=10) # Sample initial conditions of Lorenz system from uniform distribution
behaviour_sigma=1 # Standard deviation noise added to behaviour
seed=12345

In [3]:
with open(os.path.join('..', 'latentneural', 'data', 'storage', \
                       'lorenz', 'grid', 'settings_template.yaml'), 'r') as f:
   training_settings_template = yaml.safe_load(f)
print(training_settings_template)

{'data': {'behavioural_keys': {'test': 'test_behaviours', 'train': 'train_behaviours', 'validation': 'valid_behaviours'}, 'dataset_filename': 'dataset.h5', 'directory': './latentneural/data/storage/lorenz/grid/{{name}}', 'latent_keys': {'test': 'test_latent', 'train': 'train_latent', 'validation': 'valid_latent'}, 'metadata_filename': 'metadata.json', 'neural_keys': {'test': 'test_data', 'train': 'train_data', 'validation': 'valid_data'}}, 'model': {'settings': {'default_layer_settings': {'kernel_initializer': {'arguments': {'distribution': 'normal', 'mode': 'fan_in', 'scale': 1.0}, 'type': 'variance_scaling'}, 'kernel_regularizer': {'arguments': {'l': 0.1}, 'type': 'l2'}}, 'encoded_space': 64, 'layers': {'encoder': {'dropout': 0.05, 'var_max': 0.1, 'var_min': 0.1}, 'decoder': {'kernel_initializer': {'arguments': {'distribution': 'normal', 'mode': 'fan_in', 'scale': 1.0}, 'type': 'variance_scaling'}, 'kernel_regularizer': {'arguments': {'l': 1}, 'type': 'l2'}, 'original_cell': False, '

In [4]:
np.random.seed(seed)
tf.random.set_seed(seed)

for behaviour_sigma in [0.5, 1, 2]:
    for base_rate in [5, 10, 15]:
        for train_num in [50, 100, 200]:
            inner_seed = np.random.randint(0, 2**32 - 1)
            g: LorenzGenerator = LorenzGenerator()
            time, behaviour, spikes, rates, behaviour_w, neural_w, latent = \
                g.generate_spikes_and_behaviour(
                    step=step,
                    stop=stop,
                    n=neurons,
                    base=base_rate,
                    l=latent_dim,
                    b=relevant_dim,
                    y=behaviour_dim,
                    behaviour_sigma=behaviour_sigma,
                    trials=trials,
                    initial_conditions=initial_conditions,
                    conditions=1,
                    seed=inner_seed
                )

            c_i = 0
            generation_settings = dict(
                step=step,
                stop=stop,
                neurons=neurons,
                base_rate=base_rate,
                latent_dim=latent_dim,
                relevant_dim=relevant_dim,
                behaviour_dim=behaviour_dim,
                conditions=conditions,
                behaviour_sigma=behaviour_sigma,
                trials=trials,
                initial_conditions={'type': 'uniform', 'arguments': {'min': -10, 'max': 10}},
                selected_condition=c_i,
                seed=inner_seed)

            valid_num = int(train_num * 0.2)
            test_num = trials - (train_num + valid_num)

            dataset, settings = DataManager.build_dataset(
                neural_data=spikes[c_i,:,:,:],
                behaviour_data=behaviour[c_i,:,:,:],
                settings=generation_settings,
                noisless_behaviour_data=latent[c_i,:,:,-behaviour_w.shape[1]:] @ behaviour_w[c_i,:,:],
                rates_data=rates[c_i,:,:,:],
                latent_data=latent[c_i,:,:,:],
                time_data=time,
                behaviour_weights=behaviour_w[c_i,:,:],
                neural_weights=neural_w[c_i,:,:],
                train_pct=train_num,
                valid_pct=valid_num,
                test_pct=test_num
            )
            dirname = 'train_trials=%d|baselinerate=%d|behaviour_sigma=%.1f' % \
                (train_num, base_rate, behaviour_sigma)
            data_dir = os.path.join('..', 'latentneural', 'data', 'storage', 'lorenz', 'grid', \
                                    dirname)
            DataManager.store_dataset(
                dataset=dataset,
                settings=settings,
                directory=data_dir)
            logger.info('Dataset stored.')
            
            training_settings = deepcopy(training_settings_template)
            training_settings['data']['directory'] = \
                training_settings['data']['directory'].replace('{{name}}', dirname)
            training_settings['output']['directory'] = \
                training_settings['output']['directory'].replace('{{name}}', dirname)
            
            with open(os.path.join(data_dir, 'training_settings.yaml'), 'w') as fp:
                yaml.dump(training_settings, fp)
            logger.info('Training settings stored.')
            logger.info('Stored train_trials=%d|baselinerate=%d|behaviour_sigma=%.1f' % \
                (train_num, base_rate, behaviour_sigma))

[11:33:03.862] INFO [latentneural.utils.logging.store_dataset:185] Saving to ../latentneural/data/storage/lorenz/grid/train_trials=50|baselinerate=5|behaviour_sigma=0.5/dataset.h5
[11:33:03.873] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_data
[11:33:03.876] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_data
[11:33:03.877] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_data
[11:33:03.896] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours
[11:33:03.898] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours
[11:33:03.900] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours
[11:33:03.903] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours_noiseless
[11:33:03.905] INFO [latentneural.utils.logging.store

[11:33:18.578] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours
[11:33:18.579] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours
[11:33:18.583] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours_noiseless
[11:33:18.585] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours_noiseless
[11:33:18.587] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours_noiseless
[11:33:18.591] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_rates
[11:33:18.594] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_rates
[11:33:18.595] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_rates
[11:33:18.614] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_

[11:33:33.130] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_rates
[11:33:33.133] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_rates
[11:33:33.135] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_rates
[11:33:33.156] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_latent
[11:33:33.157] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_latent
[11:33:33.159] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_latent
[11:33:33.163] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: time_data
[11:33:33.164] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: relevant_dims
[11:33:33.165] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: behaviour_weights
[11:33:33.167] INFO [latentneural.utils

[11:33:47.878] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: time_data
[11:33:47.880] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: relevant_dims
[11:33:47.881] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: behaviour_weights
[11:33:47.883] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: neural_weights
[11:33:48.074] INFO [latentneural.utils.logging.<module>:66] Dataset stored.
[11:33:48.081] INFO [latentneural.utils.logging.<module>:76] Training settings stored.
[11:33:48.082] INFO [latentneural.utils.logging.<module>:77] Stored train_trials=50|baselinerate=5|behaviour_sigma=1.0
[11:33:52.649] INFO [latentneural.utils.logging.store_dataset:185] Saving to ../latentneural/data/storage/lorenz/grid/train_trials=100|baselinerate=5|behaviour_sigma=1.0/dataset.h5
[11:33:52.660] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_dat

[11:34:02.752] INFO [latentneural.utils.logging.<module>:77] Stored train_trials=50|baselinerate=10|behaviour_sigma=1.0
[11:34:07.338] INFO [latentneural.utils.logging.store_dataset:185] Saving to ../latentneural/data/storage/lorenz/grid/train_trials=100|baselinerate=10|behaviour_sigma=1.0/dataset.h5
[11:34:07.348] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_data
[11:34:07.352] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_data
[11:34:07.354] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_data
[11:34:07.371] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours
[11:34:07.373] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours
[11:34:07.375] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours
[11:34:07.378] INFO [latentneural.utils.logging.stor

[11:34:22.065] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours
[11:34:22.066] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours
[11:34:22.068] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours
[11:34:22.072] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours_noiseless
[11:34:22.074] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours_noiseless
[11:34:22.075] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours_noiseless
[11:34:22.079] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_rates
[11:34:22.082] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_rates
[11:34:22.084] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: 

[11:34:36.649] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours_noiseless
[11:34:36.653] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_rates
[11:34:36.657] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_rates
[11:34:36.659] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_rates
[11:34:36.678] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_latent
[11:34:36.680] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_latent
[11:34:36.681] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_latent
[11:34:36.685] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: time_data
[11:34:36.687] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: relevant_dims
[11:34:36.688] INFO [latentneur

[11:34:51.487] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_latent
[11:34:51.491] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: time_data
[11:34:51.492] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: relevant_dims
[11:34:51.494] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: behaviour_weights
[11:34:51.496] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: neural_weights
[11:34:51.687] INFO [latentneural.utils.logging.<module>:66] Dataset stored.
[11:34:51.694] INFO [latentneural.utils.logging.<module>:76] Training settings stored.
[11:34:51.695] INFO [latentneural.utils.logging.<module>:77] Stored train_trials=100|baselinerate=10|behaviour_sigma=2.0
[11:34:56.190] INFO [latentneural.utils.logging.store_dataset:185] Saving to ../latentneural/data/storage/lorenz/grid/train_trials=200|baselinerate=10|behaviour_sigma=2.0/datas

[11:35:06.293] INFO [latentneural.utils.logging.<module>:76] Training settings stored.
[11:35:06.294] INFO [latentneural.utils.logging.<module>:77] Stored train_trials=100|baselinerate=15|behaviour_sigma=2.0
[11:35:11.025] INFO [latentneural.utils.logging.store_dataset:185] Saving to ../latentneural/data/storage/lorenz/grid/train_trials=200|baselinerate=15|behaviour_sigma=2.0/dataset.h5
[11:35:11.034] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_data
[11:35:11.040] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_data
[11:35:11.042] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_data
[11:35:11.058] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours
[11:35:11.060] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours
[11:35:11.062] INFO [latentneural.utils.logging.store_dataset:204] Saving 